In [ ]:
import pickle
from src.utils.text_chunking import *
from sentence_transformers import util

import os
import chardet

from src.utils.retrievals import *


In [2]:
with open(r"C:\path\to\faq_embeddings", 'rb') as f:
    faq_embeddings =pickle.load(f)

In [3]:
def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        result = chardet.detect(f.read())
        return result['encoding']
    
def read_txt(file_path):
    encoding = detect_encoding(file_path)
    with open(file_path, 'r', encoding=encoding) as f:
        content = f.read()
        return content

def read_file(folder_path):
    corpus = {}
    files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]
    for file in files:
        file_path = os.path.join(folder_path, file)
        content = read_txt(file_path)
        corpus[file] = content

    return corpus

In [4]:
corpus = read_file(folder_path ="processed_data")

In [7]:
FAQ = corpus["FAQ.txt"]

In [10]:
chunking = Chunking(text=FAQ)
faq = chunking.text_chunking(FAQ, separators=["KEYWORD", "KEYWORD", "KEYWORD", "KEYWORD", "KEYWORD", "KEYWORD", "KEYWORD"], chunk_size=100, chunk_overlap=20)

In [17]:
from fcmeans import FCM
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import pandas as pd


In [27]:
fcm = FCM(n_clusters=6, m=1.1)
fcm.fit(faq_embeddings)

In [28]:
centers = fcm.centers
membership_matrix = fcm.u

In [29]:
cluster_assignment = membership_matrix.argmax(axis=1)

In [30]:
results = pd.DataFrame({
    "內文": faq,
    "分布": [membership_matrix[i] for i in range(len(membership_matrix))],
    "可能屬於": cluster_assignment
})

In [ ]:
print(f"文本分類:{results}")

In [32]:
membership_columns = [f"第{i}類" for i in range(membership_matrix.shape[1])]
final_results = pd.DataFrame(membership_matrix, columns=membership_columns)
final_results.insert(0, "文本", faq)
final_results.insert(len(final_results.columns), "可能屬於聚類", cluster_assignment)

In [34]:
final_results.to_csv("checking_cluster", index=False, encoding='utf-8')